In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def jFitnessFunction(x):
    # Load your dataset
    train_data = pd.read_csv('/content/drive/MyDrive/NF-BOT-IOT_train_preprocessed.csv')
    test_data = pd.read_csv('/content/drive/MyDrive/NF-BOT-IOT_test_preprocessed.csv')

    # Prepare training data
    X_train = train_data.iloc[:, :-1].values
    y_train = train_data.iloc[:, -1].values

    # Prepare test data
    X_test = test_data.iloc[:, :-1].values
    y_test = test_data.iloc[:, -1].values

    # Split a small portion of training data for optimization (Poor and Rich Optimization)
    X_t, _, y_t, _ = train_test_split(X_train, y_train, train_size=0.01, random_state=7)
    X_test_t, _, y_test_t, _ = train_test_split(X_test, y_test, train_size=0.01, random_state=7)

    # Define custom Poor and Rich Optimization (PARO) algorithm for feature selection
    num_features = X_t.shape[1]
    poor_agents = np.ones(num_features)  # Initialize motivation levels for poor agents
    rich_agent = np.random.choice([True, False], size=num_features)  # Randomly initialize the rich agent's selection
    best_feature_set = rich_agent
    best_accuracy = evaluate(X_t, y_t, best_feature_set)

    # Define parameters
    num_agents = 2
    max_iter = 2

    # Start the optimization process
    for iteration in range(max_iter):
        # Poor agents
        for agent in range(num_agents):
            # Generate a new candidate solution for poor agents by flipping a random feature
            new_agent = poor_agents.copy()
            flip_idx = np.random.randint(num_features)
            new_agent[flip_idx] = not new_agent[flip_idx]

            # Evaluate the new candidate solution
            new_accuracy = evaluate(X_t, y_t, new_agent)
            delta_accuracy = new_accuracy - best_accuracy

            # Update poor agent's selection based on improvement
            if delta_accuracy > 0:
                poor_agents = new_agent.copy()
                best_accuracy = new_accuracy
                best_feature_set = new_agent.copy()

        # Rich agent
        # Generate a new candidate solution for rich agent by flipping a random feature
        flip_idx = np.random.randint(num_features)
        rich_agent[flip_idx] = not rich_agent[flip_idx]

        # Evaluate the new candidate solution
        new_accuracy = evaluate(X_t, y_t, rich_agent)
        delta_accuracy = new_accuracy - best_accuracy

        # Update rich agent's selection based on improvement
        if delta_accuracy > 0:
            best_accuracy = new_accuracy
            best_feature_set = rich_agent.copy()

    # Evaluate the selected features on the test data
    selected_features = np.where(best_feature_set)[0]
    X_test_selected = X_test[:, selected_features]

    # Train and evaluate the classifier
    classifier = RandomForestClassifier()
    classifier.fit(X_t[:, selected_features], y_t)
    y_pred = classifier.predict(X_test_selected)

    # Calculate accuracy
    accuracy = np.sum(y_pred == y_test_t) / len(y_test_t)

    # Assign fitness as negative of accuracy (since it's a minimization problem)
    fitness = -accuracy
    return fitness

def evaluate(X, y, selected_features):
    # Evaluate the selected features
    X_subset = X[:, selected_features]

    # Train and evaluate the classifier
    classifier = RandomForestClassifier()
    classifier.fit(X_subset, y)
    y_pred = classifier.predict(X_subset)

    # Calculate accuracy
    accuracy = np.sum(y_pred == y) / len(y)
    return accuracy
